# Heidelberg Tutorial

In this tutorial we will be learning how to use NeuroTorch to train a neural network to recognize audio recordings of spoken digits.

## Setup

You can install the dependencies by running the following commands:

In [2]:
!pip install -r heidelberg_requirements.txt
!pip install git+https://github.com/JeremieGince/PythonBasicTools

  Cloning https://github.com/JeremieGince/PythonBasicTools to c:\users\gince\appdata\local\temp\pip-req-build-w2nbkj8c
  Resolved https://github.com/JeremieGince/PythonBasicTools to commit 63b55f13565762b20503a55c3abf4105e0f28064
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached matplotlib-3.6.0-cp39-cp39-win_amd64.whl (7.2 MB)
  Using cached pandas-1.5.0-cp39-cp39-win_amd64.whl (10.9 MB)
  Using cached scikit_learn-1.1.2-cp39-cp39-win_amd64.whl (7.4 MB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Using cached fonttools-4.37.3-py3-none-any.whl (959 kB)
  Using cached contourpy-1.0.5-cp39-cp39-win_amd64.whl (161 kB)
  Using cached kiwisolver-1.4.4-cp39-cp39-win_amd64.whl (55 kB)
  Usi

  Running command git clone --filter=blob:none -q https://github.com/JeremieGince/PythonBasicTools 'C:\Users\gince\AppData\Local\Temp\pip-req-build-w2nbkj8c'
You should consider upgrading via the 'E:\Github\NeuroTorch\tutorials\heidelberg\heidelberg_venv\Scripts\python.exe -m pip install --upgrade pip' command.


  Running command git clone --filter=blob:none -q https://github.com/JeremieGince/PythonBasicTools 'C:\Users\gince\AppData\Local\Temp\pip-req-build-kt7nlojz'
You should consider upgrading via the 'E:\Github\NeuroTorch\tutorials\heidelberg\heidelberg_venv\Scripts\python.exe -m pip install --upgrade pip' command.



  Cloning https://github.com/JeremieGince/PythonBasicTools to c:\users\gince\appdata\local\temp\pip-req-build-kt7nlojz
  Resolved https://github.com/JeremieGince/PythonBasicTools to commit 63b55f13565762b20503a55c3abf4105e0f28064
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


In [ ]:
!pip install git+https://github.com/JeremieGince/NeuroTorch.git

If you have a cuda device and want to use it for this tutorial (it is recommended to do so), you can uninstall pytorch with `pip uninstall torch` and re-install it with the right cuda version by generating a command with [PyTorch GetStarted](https://pytorch.org/get-started/locally/) web page.

After setting up the virtual environment, we will need to import the necessary packages.

In [1]:
import os
import pprint
from collections import OrderedDict

import torch
from pythonbasictools.device import log_device_setup, DeepLib
from pythonbasictools.logging import logs_file_setup

from dataset import get_dataloaders
import neurotorch as nt
from neurotorch import Dimension, DimensionProperty
from neurotorch.modules import HeavisideSigmoidApprox
from neurotorch.callbacks import CheckpointManager, LoadCheckpointMode
from neurotorch.metrics import ClassificationMetrics
from neurotorch.modules import SequentialModel
from neurotorch.modules.layers import SpyLIFLayer, SpyLILayer
from neurotorch.trainers import ClassificationTrainer
from neurotorch.regularization import RegularizationList, L2, L1

In [2]:
logs_file_setup("heidelberg_tutorial", add_stdout=False)
log_device_setup(deepLib=DeepLib.Pytorch)
if torch.cuda.is_available():
	torch.cuda.set_per_process_memory_fraction(0.8)

INFO:root:Logs file at: .//logs/logs-20-09-2022/heidelberg_tutorial-16637289759816937.log

INFO:root:__Python VERSION: 3.9.12 (tags/v3.9.12:b28265d, Mar 23 2022, 23:52:46) [MSC v.1929 64 bit (AMD64)]
INFO:root:Number of available cores: 12.
INFO:root:Number of available logical processors: 24.
INFO:root:__pyTorch VERSION:1.12.1+cu116
INFO:root:__CUDA VERSION:
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Nov_30_19:15:10_Pacific_Standard_Time_2020
Cuda compilation tools, release 11.2, V11.2.67
Build cuda_11.2.r11.2/compiler.29373293_0

INFO:root:__nvidia-smi: Not Found
INFO:root:__CUDNN VERSION:8302
INFO:root:__Number CUDA Devices:1
INFO:root:
-------------------------
DEVICE: cuda
-------------------------

INFO:root:NVIDIA GeForce RTX 3070
INFO:root:Memory Usage:
INFO:root:Allocated: 0.0 GB
INFO:root:Cached:   0.0 GB
INFO:root:Memory summary: 
|===========================================================================|
|                

## Initialization

It's the time to defined our training and networks parameters. In the following,

- the number of iteration is the number of time the trainer will pass through the entire training dataset;
- the batch size is the number of samples that will be loaded at the same time for each forward and backward pass;
- the learning rate is the learning rate used by the optimizer;
- the random seed is the seed used by the random number generator;
- the number of steps is the number of euler integration steps performed by the model during each forward and backward pass;
- the number of hidden neurons is the number of neurons that will be used in the hidden layer of the model;
- dt is the time step of the euler integration;

In [3]:
# Training parameters
n_iterations = 50
batch_size = 256
learning_rate = 2e-4
seed = 42

# Network parameters
dynamic_type = nt.SpyLIFLayer
n_steps = 100
n_hidden_neurons = 256
dt = 1e-3

In [4]:
torch.manual_seed(seed)

Here we're initializing a callback of the trainer used to save the network during the training.

In [5]:
checkpoint_folder = f"./checkpoints/heidelberg_{dynamic_type.__name__}"
checkpoint_manager = CheckpointManager(checkpoint_folder, metric="val_accuracy", minimise_metric=False, save_best_only=True, start_save_at=int(n_iterations * 0.9))


Here is an exemple of the Heidelberg dataset:
![Heidelberg_exemples](images/heidelberg_exemples.png)

In [6]:
dataloaders = get_dataloaders(
	batch_size=batch_size,
	train_val_split_ratio=0.95,
)

We can finally define our model. The sequential model is a neural network that is composed of a list of layers. The list of layers is ordered and the first layer is the input layer. The last layer is the output layer. The sequential model can be found in the subpackage `neurotorch.modules`. Note that we can specify the input transform that will be applied to the input data before the forward pass through the list of layers. If the transformation is computationally expensive, it is recommended to put it in the dataloaders instead of the model for the training. Usually the input transform in the model is used when the model is used for inference.

In [7]:
network = SequentialModel(
	layers=[
		dynamic_type(
			input_size=nt.Size([
				Dimension(None, DimensionProperty.TIME),
				Dimension(dataloaders["test"].dataset.n_units, DimensionProperty.NONE)
			]),
			output_size=n_hidden_neurons,
			use_recurrent_connection=True,
			spike_func=HeavisideSigmoidApprox,
			dt=dt,
			# spikes_regularization_factor=1e-6,
		),
		SpyLILayer(dt=dt, output_size=dataloaders["test"].dataset.n_classes),
	],
	name=f"heidelberg_network",
	checkpoint_folder=checkpoint_folder,
)

After the instantiation of the model, we have to build it, so it can infer the missing sizes of the layers in the model and create the tensors that will be used during the training.

In [8]:
network.build()
print(network)

SequentialModel(
  (_to_device_transform): ToDevice(cuda, async=True)
  (input_layers): ModuleDict(
    (input_0): SpyLIFLayer<input_0>(700<->256)[LearningType.BPTT]@cuda
  )
  (hidden_layers): ModuleList()
  (output_layers): ModuleDict(
    (output_0): SpyLILayer<output_0>(256->20)[LearningType.BPTT]@cuda
  )
  (input_transform): ModuleDict(
    (input_0): Sequential(
      (0): CallableToModuleWrapper(Compose(
          ToTensor()
      ))
      (1): ToDevice(cuda, async=True)
    )
  )
  (output_transform): ModuleDict(
    (output_0): IdentityTransform()
  )
)


In [9]:
regularization = RegularizationList([
	L2(network.parameters(), Lambda=1e-5),
	L1(network.parameters(), Lambda=1e-6),
])

## Training

In the next cell, we create the trainer that will be used to trained our network. They are several types of trainers that can be found in the subpackage `neurotorch.trainers` like the `ClassificationTrainer`, the `RegressionTrainer` or the `PPOTrainer`. In this tutorial, we will use the `ClassificationTrainer` because we are trying to classify some images. Note that the callbacks are some object that will be called during the training. They are found in the subpackage `neurotorch.callbacks` like the `CheckpointManager`.

In [10]:
trainer = ClassificationTrainer(
	model=network,
	# regularization=regularization,
	optimizer=torch.optim.Adam(network.parameters(), lr=learning_rate, weight_decay=0.0),
	# regularization_optimizer=torch.optim.SGD(regularization.parameters(), lr=1e-2, weight_decay=0.0),
	callbacks=checkpoint_manager,
	verbose=True,
)

The training will start! Let's see how our simple network will perform on this task.

Note: the argument `force_overwrite` is used to overwrite the existing checkpoint if it exists and the argument `load_checkpoint_mode` if the model will be loaded from the last checkpoint (`LoadCheckpointMode.LAST_ITR`) or from the best checkpoint (`LoadCheckpointMode.BEST_ITR`). The loading mode is really useful when you want to stop the training and restart it later.

In [ ]:
training_history = trainer.train(
	dataloaders["train"],
	dataloaders["val"],
	n_iterations=n_iterations,
	load_checkpoint_mode=LoadCheckpointMode.LAST_ITR,
	# force_overwrite=True,
)

Training:   0%|          | 0/50 [00:00<?, ?itr/s]

The training is finished! Let's see how our network learned over the iterations.

Note: the next graphs are really useful to see if the model overfit or underfit the data over the iterations. It's why you can use the callback `TrainingHistoryVisualizationCallback` to see those graphs in real time.

In [ ]:
training_history.plot(show=True)

We can load the model at the best or the last iteration for the testing phase.

In [ ]:
network.load_checkpoint(checkpoint_manager.checkpoints_meta_path, LoadCheckpointMode.BEST_ITR, verbose=True)

We can now see how our network perform on the test dataset.

In [ ]:
predictions = {
	k: ClassificationMetrics.compute_y_true_y_pred(network, dataloader, verbose=True, desc=f"{k} predictions")
	for k, dataloader in dataloaders.items()
}

In [ ]:
accuracies = {
	k: ClassificationMetrics.accuracy(network, y_true=y_true, y_pred=y_pred)
	for k, (y_true, y_pred) in predictions.items()
}
pprint.pprint(accuracies)

Let's see the results on other popular classification metrics.

In [ ]:
precisions = {
	k: ClassificationMetrics.precision(network, y_true=y_true, y_pred=y_pred)
	for k, (y_true, y_pred) in predictions.items()
}
recalls = {
	k: ClassificationMetrics.recall(network, y_true=y_true, y_pred=y_pred)
	for k, (y_true, y_pred) in predictions.items()
}
f1s = {
	k: ClassificationMetrics.f1(network, y_true=y_true, y_pred=y_pred)
	for k, (y_true, y_pred) in predictions.items()
}

In [ ]:
results = OrderedDict(dict(
	network=network,
	accuracies=accuracies,
	precisions=precisions,
	recalls=recalls,
	f1s=f1s,
))

In [ ]:
pprint.pprint(results, indent=4)

This is the end of the tutorial!

# Conclusion

NeuroTorch is a library that allows you to easily build neural networks and train them. Moreover, it allows you to do audio signal classification using dynamics from neurosciences.